### Evaluación II


En esta evaluación nos enfrentamos a un desafío emocionante de trabajar en un proyecto real para una empresa que realiza un estudio de universidades por el mundo. 

El proyecto tiene como objetivo identificar todas las universidades ubicadas en tres países específicos:

    - Estados Unidos
    - Canadá
    - Argentina

Para llevar a cabo esta tarea, utilizaremos la API de "Universities Hipolabs", una fuente confiable y completa de información sobre las universidades en todo el mundo. 

Con la ayuda de esta API, podemos acceder a una gran cantidad de datos relevantes, incluyendo el nombre de la universidad, la ciudad donde esta ubicada, el nombre de la institución y otra información importante que nos permitirá llevar a cabo un análisis detallado.


Es importante tener en cuenta que este proyecto requerirá un conocimiento profundo de herramientas y técnicas de análisis de datos, así como habilidades en programación y manejo de APIs. 

También es importante tener una comprensión sólida de la estructura y organización de los datos, ya que esto nos permitirá hacer preguntas importantes y obtener respuestas significativas a partir de los datos.


En resumen, esta prueba técnica ofrece una excelente oportunidad para demostrar habilidades y conocimientos en análisis de datos y programación, mientras se trabaja en un proyecto real y relevante para una empresa. 

Al finalizar del proyecto, esperamos obtener información valiosa que ayudará a la empresa a tomar decisiones más informadas sobre las universidades en los tres países objetivo.

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 

import requests
import pandas as pd
import numpy as np

import mysql.connector
from mysql.connector import errorcode

from geopy.geocoders import Nominatim

pd.options.display.max_columns = None

ModuleNotFoundError: No module named 'requests'

1. Utilizando la API extraed toda la información que podáis de ella. 

    La url para hacer las llamadas es:
    API_URL = "http://universities.hipolabs.com/search?country=NOMBREPAIS"

In [343]:
#Hacemos una lista de los paises que nos interesan

lista_paises = ['United States', 'Canada', 'Argentina']

In [344]:
#Creamos un dataframe vacío
df = pd.DataFrame()

#iterando por la lista de paises vamos llamando a la API y adjuntamos los resultados al dataframe vacío
for pais in lista_paises:
    url = f'http://universities.hipolabs.com/search?country={pais}'
    response = requests.get(url=url)
    response.status_code
    response.reason
    df_pais = pd.DataFrame(response.json())
    df = pd.concat([df, df_pais], axis = 0, ignore_index = True)

200

'OK'

200

'OK'

200

'OK'

In [345]:
#visualizamos el dataframe resultante

df.sample(10)

,web_pages,alpha_two_code,state-province,name,domains,country
936,[http://www.union.edu/],US,None,Union College,[union.edu],United States
3774,[http://www.kcc.edu],US,None,Kankakee Community College,[kcc.edu],United States
3076,[http://www.saic.edu],US,None,School of the Art Institute of Chicago,[saic.edu],United States
3255,[http://www.udmercy.edu/],US,None,University of Detroit Mercy,[udmercy.edu],United States
3982,[http://www.wp.missouristate.edu],US,None,Missouri State University-​West Plains,[wp.missouristate.edu],United States
98,[http://www.ccnn.edu/],US,None,Career College of Northern Nevada,[ccnn.edu],United States
4126,[http://www.mayland.edu],US,None,Mayland Community College,[mayland.edu],United States
3640,[http://www.missioncollege.edu/],US,None,Mission College,[missioncollege.edu],United States
77,[http://www.njcu.edu/],US,None,New Jersey City University,[njcu.edu],United States
811,[http://www.seattleu.edu/],US,None,Seattle University,[seattleu.edu],United States


2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:

    -Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.

    -La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.

In [347]:
#Lo primero que vamos a hacer es crear un diccionario con los nombres de las columnas y los nuevos nombres que queremos que tengan

nuevas_columnas = {col:col.replace('-', '_') for col in df.columns}
nuevas_columnas

{'web_pages': 'web_pages',
 'alpha_two_code': 'alpha_two_code',
 'state-province': 'state_province',
 'name': 'name',
 'domains': 'domains',
 'country': 'country'}

In [348]:
#Cambiamos los nombres de las columnas

df.rename(columns = nuevas_columnas, inplace = True)

In [349]:
#Quitamos la columna 'domains'

df.drop('domains', axis = 1, inplace = True)

3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. 

    Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejercicio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva.

In [350]:
#Comprobamos cómo están separados los distintos valores

df[df['name']== 'Cégep de Saint-Jérôme']

,web_pages,alpha_two_code,state_province,name,country
4536,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",CA,Quebec,Cégep de Saint-Jérôme,Canada
4686,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",CA,Quebec,Cégep de Saint-Jérôme,Canada


In [351]:
#Para poder aplicar el método str.split() los datos deben ser de tipo string

df['web_pages'] = df['web_pages'].astype(dtype = 'str', errors = 'raise')


In [352]:
#Convertimos los string en listas

df['web_pages'] = df['web_pages'].str.split(',')

In [353]:
#aplicamos el método explode

df = df.explode('web_pages')

4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [354]:
#Comprobamos si tenemos duplicados

df.duplicated(subset = ['name']).sum()

2543

In [355]:
#Eliminamos los duplicados. Nos quedamos con el primer registro de cada uno de los duplicados

df = df.drop_duplicates(['name'])

In [356]:
#Terminamos de limpiar la columna 'web_pages'

df['web_pages'] = df['web_pages'].str.replace('[', '').str.replace(']', '')


/tmp/ipykernel_17965/3079133702.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['web_pages'] = df['web_pages'].str.replace('[', '').str.replace(']', '')


5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [357]:
def nulos(dato):
    """
        Funcion que recibe un None y lo convierte a NaN
        Args:
            dato: es un None
        Returns: 
            NaN
    """
    if dato is None:
        return np.nan
    else:
        return dato

In [358]:
#Aplicamos la función que hemos creado a la columna del dataframe

df['state_province'] = df['state_province'].apply(nulos)

6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [359]:
df = df.fillna('Unknown')

7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. 

    Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso
    
    Para desarrollar este ejercicio deberéis:

    -Sacar los valores únicos de la columna state_province.

    -Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:

        NV: reemplazalo por Nevada
        TX: reemplazalo por Texas
        IN: reemplazalo por Indianapolis
        CA: reemplazalo por California
        VA: reemplazalo por Virginia
        NY: reemplazalo por New York
        MI: reemplazalo por Michigan
        GA: reemplazalo por Georgia
        ND: reemplazalo por North Dakota

In [360]:
#Creamos un diccionario con los valores antiguos y los nuevos

dict1 = {'NV' : 'Nevada',
        'TX' : 'Texas',
        'IN' : 'Indianapolis',
        'CA' : 'California',
        'VA' : 'Virginia',
        'NY' : 'New York',
        'MI' : 'Michigan',
        'GA' : 'Georgia',
        'ND' : 'North Dakota'}

In [361]:
#Sustituimos los valores

df['state_province'].replace(dict1, inplace = True)

    -Otros valores que tenemos más formateados son y que deberemos reemplazar:

        -New York, NY. Deberéis reemplazarlo por "New York"
        
        -'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires" 

In [362]:

dict2 = {'New York, NY': 'New York',
         'Ciudad Autónoma de Buenos Aires' : 'Buenos Aires'}

In [363]:
df['state_province'].replace(dict2, inplace = True)

    -Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

In [364]:
#Algunos nombres nos devuelven localizaciones de otros paises así que los cambiamos
dict3 = {'Georgia' : 'Georgia, USA',
         'Córdoba' : 'Cordoba, Argentina',
         'Formosa' : 'Formosa, Argentina',
         'Santa Fe' : 'Santa Fe, Argentina',
         'La Rioja' : 'La Rioja, Argentina',
         'San Juan' : 'San Juan, Argentina',
         'San Luis' : 'San Luis, Argentina'}

df['state_province'].replace(dict3, inplace = True)

In [365]:
provincias = df['state_province'].unique()
provincias

array(['Unknown', 'Pennsylvania', 'Texas', 'Utah', 'Nevada', 'Iowa',
       'Virginia', 'Indiana', 'Colorado', 'Ohio', 'Indianapolis',
       'New York', 'California', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Washington', 'Missouri',
       'North Dakota', 'Michigan', 'Florida', 'Georgia, USA', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Entre Ríos', 'Salta', 'Cordoba, Argentina',
       'Mendoza', 'Santa Fé', 'Santiago Del Estero', 'Misiones',
       'Catamarca', 'Formosa, Argentina', 'Jujuy', 'La Rioja, Argentina',
       'La Pampa', 'San Juan, Argentina', 'San Luis, Argentina',
       'Tucumán'], dtype=object)

    -Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

In [366]:
geo = Nominatim(user_agent = 'catalina')
df_local = pd.DataFrame()

for prov in provincias:
        
    localizacion = geo.geocode(prov)
    
    if prov == 'Unknown':
        df_prov = pd.DataFrame([localizacion.raw['name'], 'Unknown', 'Unknown']).T
        df_local = pd.concat([df_local, df_prov], axis = 0)
                               
    else:
        df_prov = pd.DataFrame([localizacion.raw['name'], localizacion.raw['lat'], localizacion.raw['lon']]).T
        df_local = pd.concat([df_local, df_prov], axis = 0)      
    
df_local.set_axis(['state_province', 'latitude', 'longitude'], axis = 'columns', inplace = True)
df_local.head()

,state_province,latitude,longitude
0,Unknown,Unknown,Unknown
0,Pennsylvania,40.9699889,-77.7278831
0,Texas,31.2638905,-98.5456116
0,Utah,39.4225192,-111.714358
0,Nevada,39.5158825,-116.8537227


    -Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [367]:
#Ahora ya podemos juntar los dataframes

df_final = df.merge(df_local, on = 'state_province')
df_final.head(2)

,web_pages,alpha_two_code,state_province,name,country,latitude,longitude
0,'http://www.marywood.edu',US,Unknown,Marywood University,United States,Unknown,Unknown
1,'http://www.lindenwood.edu/',US,Unknown,Lindenwood University,United States,Unknown,Unknown


8. Crea una BBDD en mysql que contenga las siguientes tablas:

    Tabla países: donde encontraremos las siguientes columnas:

        . idestado: primary key, integer, autoincremental
        . nombre_pais: varchar
        . nombre_provincia: varchar
        . latitud: decimal
        . longitud: decimal
    Tabla universidades: donde encontraremos las siguientes columnas:
    
        . iduniversidades: primary key, integer, autoincremental
        . nombre_universidad: varchar
        . pagina_web: varchar
        . paises_idestado: foreing key

In [3]:
#Conectamos con la base de datos

mydb = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="AlumnaAdalab", 
        auth_plugin = 'mysql_native_password')
   
mycursor = mydb.cursor()

In [6]:
#creamos la base de datos

mycursor.execute("CREATE DATABASE IF NOT EXISTS `Universidades`")

In [7]:
#creamos las tablas

mycursor.execute =('''
CREATE TABLE IF NOT EXISTS `universidades`.`provincias` (
  `id_provincia` INT NOT NULL AUTO_INCREMENT,
  `nombre_provincia` VARCHAR,
  `nombre_pais` VARCHAR,
  `latitud` DECIMAL,
  `longitud` DECIMAL,
  PRIMARY KEY (`id_provincia`),   
  ENGINE = InnoDB;''')

In [8]:
mycursor.execute = ('''
CREATE TABLE IF NOT EXISTS `universidades`.`universidades` (
  `id_universidad` INT NOT NULL AUTO_INCREMENT,
  `nombre_universidad` VARCHAR,
  `pagina_web` VARCHAR,
  `id_provincia` INT,                                                            
  PRIMARY KEY (`id_universidad`),                       
  CONSTRAINT `fk_provincias_universidades`                   
    FOREIGN KEY (`id_provincia`)                                                  
    REFERENCES `universidades`.`provincias` (`id_provincia`),
ENGINE = InnoDB;''')

9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs

In [ ]:
# creamos la funcion que nos permite conectarnos a la API y obtener datos

def conexion_API(paises):
    """
        Funcion que nos permite conectar con la API
        y obtener un dataframe con los datos solicitados
        Args:
            paises: List
        Returns: dataframe
            
    """
    df = pd.DataFrame()
    for pais in paises:
        url = f'http://universities.hipolabs.com/search?country={pais}'
        response = requests.get(url=url)
        response.status_code
        response.reason
        df_pais = pd.DataFrame(response.json())
        df = pd.concat([df, df_pais], axis = 0, ignore_index = True)
        
    return df

In [ ]:
#creamos la funcion de limpieza del dataframe

def limpieza(df):
    """
        Funcion que nos permite limpiar un dataframe
        Unificando nombre de columnas
        Borrar columna 'domains' redundante
        Separa informacion de la columna 'web_pages'
        Eliminar duplicados
        Args:
            df: dataframe
            
        Returns: dataframe limpio
            
    """
    
    df.rename(columns = {col:col.replace('-', '_') for col in df.columns}, inplace = True)
    df.drop('domains', axis = 1, inplace = True)
    df['web_pages'] = df['web_pages'].astype(dtype = 'str', errors = 'raise')
    df['web_pages'] = df['web_pages'].str.split(',')
    df = df.explode('web_pages')
    df = df.drop_duplicates(['name'])
    df['web_pages'] = df['web_pages'].str.replace('[', '').str.replace(']', '')
    
    
    return df
    

In [ ]:
#funcion para el tratamiento de nulos del dataframe

def limpiar_nulos(df):
    """
        Funcion que nos permite imputar los nulos del dataframe
        Args:
            df: dataframe
            
        Returns: dataframe limpio
            
    """
    for indice in range(df.shape[0]):
        if df['state_province'][indice] is None:
            df['state_province'][indice] = np.nan
        else:
            continue
    
    df = df.fillna('Unknown')
    
    return df
    
    

In [ ]:
#funcion para añadir las coordenadas de las universidades

def coordenadas(user_agent, lista_provincias, df_universidades):
    """
        Funcion que nos permite obtener las coordenadas de las provincias en las que 
        estan las universidades usando geopy
        Args:
            user_agent: String
            lista_provincias: lista
            df_universidades: dataframe
        Returns: dataframe con las coordenadas añadidas
            
    """
    geo = Nominatim(user_agent = 'catalina')
    df_local = pd.DataFrame()    

    for prov in provincias:
            
        localizacion = geo.geocode(prov)
        
        if prov == 'Unknown':
            df_prov = pd.DataFrame([localizacion.raw['name'], 'Unknown', 'Unknown']).T
            df_local = pd.concat([df_local, df_prov], axis = 0)
                                
        else:
            df_prov = pd.DataFrame([localizacion.raw['name'], localizacion.raw['lat'], localizacion.raw['lon']]).T
            df_local = pd.concat([df_local, df_prov], axis = 0)      
        
    df_local.set_axis(['state_province', 'latitude', 'longitude'], axis = 'columns', inplace = True)
    df_local.head()
    
    df_final = df_universidades.merge(df_local, on = 'state_province')
    
    return df_final
    

In [ ]:
#creamos la funcion para crear una base de datos

def crear_bbdd(nombre_bbdd, contraseña):
    """
        Funcion que nos permite conectar con un servidor
        MySQL y crear una nueva base de datos
        Args:
            nombre_bbdd: String
            contraseña: string
        Returns: cursor
            
    """
    mydb = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password=contraseña,
        auth_plugin = 'mysql_native_password') 
    print("Conexión realizada con éxito")    
    mycursor = mydb.cursor()    
    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [ ]:
#creamos la funcion para crear e insertar tablas

def crear_insertar_tabla(nombre_bbdd, contraseña, query): 
    """
        Funcion que nos permite conectar con un servidor
        MySQL y ejecutar una query
        Args:
            nombre_bbdd: string
            contraseña: string
            query: string
        Returns: sin return
            
    """
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}",
                                     auth_plugin = 'mysql_native_password')
   
    mycursor = cnx.cursor() 
    try:
        mycursor.execute(query)
        cnx.commit()
    
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

10. BONUS

    -Introduce los datos en la BBDD de SQL.
    
    -Crea una clase con todo el código generado en esta evaluación.